In [ ]:
HTML(read(open("style.css"), String))

# Memoization

This notebook contains alle functions needed for memoizing function calls that we have already calculated. This will greatly reduce the runtime of the program.  

```
function memoize(f)
    global gCache
    function f_memoized(b, s, d)    # TODO: replace (b, s, d) with args... to make function generic
        if (b, s, d) in keys(gCache)
            return gCache[args]
        end
        result = f(b, s, d)
        merge!(gCache, Dict((b, s, d) => result))
        return result
    end
    return f_memoized
end
```

In [ ]:
using Chess
using NBInclude

In [ ]:
@nbinclude("ZobristHashing.ipynb")

The function `initCache()` returns an empty Dictionary of type `Dict{UInt64, Tuple{String, Int64, Int64}}` which will be used for the global Cache variable.

In [ ]:
function initCache()
    return Dict{UInt64, Tuple{String, Int64, Int64}}()
end

The `evaluate` function adds memoization to the `Max` and `Min` function. 
This function is used to evaluate the given board state using the provided evaluation function (f) and cache. It performs alpha-beta pruning to reduce the number of nodes that need to be evaluated, and checks the cache to see if the given state has already been evaluated before.

It takes in 4 arguments and 2 optional arguments.
1. `State` is the current state represented by a `Board`
1. `f` takes in either the function `alphaBetaMax` and `alphaBetaMin`
1. `score` is the static centipawn evaluation of the `state`
1. `depth` is the number of halfmoves the engine should analyze before terminating
1. `alpha` is optional and is default to -Infinity. Alpha is a minimal value that has been calculated during the recursive process
1. `beta`  is optional and is default to Infinity . Beta is a maximal value that has been calculated during the recursive process

The function `evaluate` evaluates the same result as the function `f`. Additionally it saves calculated results in the `Cache`. And uses any entries in the `Cache` if the same function has already been called.

In [ ]:
function evaluate(aBoard::AdvBoard, f::Function, depth::Int64, cache::Dict{UInt64, Tuple{String, Int64, Int64}},
                  alpha::Int64, beta::Int64, flagQuiesce::Bool=false)::Int64
    if haskey(cache, aBoard.hash)
        flag, v, d = cache[aBoard.hash]
        if d >= depth 
            if ((flag == "=") || (flag == "<=" && v <= alpha) || (flag == ">=" && beta <= v))
                return v
            elseif flag == "<="
                beta = min(beta, v)
            elseif flag == ">="
                alpha = max(alpha, v)
            end
        end
    end
    # no value stored in gCache for State or depth of stored State has less depth than required
    if flagQuiesce
        v = f(aBoard, depth, cache, alpha, beta, flagQuiesce)
    else
        v = f(aBoard, depth, cache, alpha, beta)
    end
    store_cache(aBoard.hash, depth, cache, alpha, beta, v)
    return v
end

The `store_cache` is a helping function that stores values into the `Cache`. 

It takes 5 arguments:
1. `State` is the current `state` of type `Board`
1. `depth` is the number of halfmoves the engine should analyze before terminating
1. `alpha` is optional and is default to -Infinity. Alpha is a minimal value that has been calculated during the recursive process
1. `beta`  is optional and is default to Infinity . Beta is a maximal value that has been calculated during the recursive process
1. `v` is the value that got calculated by the `evaluate` function that needs to be stored


In [ ]:
function store_cache(hash::UInt64, depth::Int64, 
                     cache::Dict{UInt64, Tuple{String, Int64, Int64}}, alpha::Int64, beta::Int64, v::Int64)
    if v <= alpha
        cache[hash] = ("<=", v, depth)
    elseif v < beta
        cache[hash] = ("=", v, depth)
    else # beta <= v
        cache[hash] = (">=", v, depth)
    end
end

In [ ]:
function updateBoardData(board::Board, score::Int64, hash::UInt64, move::Move)
    next_score = evaluate_move(board, move, score)
    next_hash = zobrist_hash(board, hash, move)
    return next_score, next_hash
end